In [18]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, TimeSeriesSplit
import numpy as np
#from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

In [19]:
# Define the path to the CSV file
file_path = './data/sales_train_evaluation.csv'

# Read the CSV file into a DataFrame
sales_df = pd.read_csv(file_path)


In [20]:
# Define the path to the CSV file
file_path = './data/calendar.csv'

# Read the CSV file into a DataFrame
calendar_df = pd.read_csv(file_path)


In [21]:
sales_df.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0


In [22]:
sales_df.shape

(30490, 1947)

In [23]:
# Reshaping the data from wide to long format, as to use days easier

sales_long = sales_df.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='day',
    value_name='sales'
)

In [53]:
sales_long.shape

(59181090, 8)

In [24]:
sales_long["cat_id"].value_counts()

cat_id
FOODS        27892170
HOUSEHOLD    20322270
HOBBIES      10966650
Name: count, dtype: int64

In [25]:
hobbies_sales_long = sales_long[sales_long['cat_id'] == 'HOBBIES']

In [26]:
hobbies_sales_long.shape

(10966650, 8)

In [27]:
ca_hobbies_sales_long = hobbies_sales_long[hobbies_sales_long['state_id'] == 'CA']

In [28]:
ca_hobbies_sales_long.shape

(4386660, 8)

In [29]:
## All SKU in Hobbie Department, within Primary Store in California
# About 1.1 M entries

alpha_ca_hobbies_sales_long = ca_hobbies_sales_long[ca_hobbies_sales_long['store_id'] == 'CA_1']

In [30]:
alpha_ca_hobbies_sales_long.shape

(1096665, 8)

In [31]:
# Assume 'df' is your DataFrame and 'column_name' is the name of the column you want to extract
list_hobbies_products = alpha_ca_hobbies_sales_long['item_id'].tolist()

#list_hobbies_products

In [32]:
alpha_ca_hobbies_sales_long

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59151160,HOBBIES_2_145_CA_1_evaluation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_1,CA,d_1941,0
59151161,HOBBIES_2_146_CA_1_evaluation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_1,CA,d_1941,0
59151162,HOBBIES_2_147_CA_1_evaluation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_1,CA,d_1941,0
59151163,HOBBIES_2_148_CA_1_evaluation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_1,CA,d_1941,0


In [33]:
## Spitting out filtered_sales_df containing each Product in Hobbies Department
# Uncomment to generate

#alpha_ca_hobbies_sales_long.to_csv('hobbies_dep_sales.csv', index=True)

In [34]:
alpha_ca_hobbies_sales_long["item_id"].nunique()

565

______________________________